<a href="https://colab.research.google.com/github/zarkuslol/gsi073-llm/blob/main/Rede_Neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importações

In [4]:
import torch
import sklearn
from torch import nn

# Modelo
import torch.nn.functional as F

# Carregamento dos dados

In [2]:
iris = sklearn.datasets.load_iris()
X = iris.data
y = (iris.target == 1).astype(float)

In [8]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

/tmp/ipython-input-2694997208.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)
/tmp/ipython-input-2694997208.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float32).view(-1, 1)


# Definição do modelo

In [5]:
class NeuralNet(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(NeuralNet, self).__init__()
    self.fc1 = nn.Linear(input_dim, hidden_dim)
    self.fc2 = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

# Criação do modelo

In [6]:
model = NeuralNet(4, 8, 1)

import copy
clone = copy.deepcopy(model)

learning_rate = 0.1

loss_function = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Loop de treino com SGD

In [9]:
for epoch in range(1000):
  optimizer.zero_grad()
  outputs = model(X)
  loss = loss_function(outputs, y)
  loss.backward()
  optimizer.step()

  if (epoch + 1) % 100 == 0:
    print(f"Época [{epoch+1}/100], Loss: {loss.item():.4f}")

Época [100/100], Loss: 0.4795
Época [200/100], Loss: 0.3958
Época [300/100], Loss: 0.2987
Época [400/100], Loss: 0.2177
Época [500/100], Loss: 0.1362
Época [600/100], Loss: 0.2743
Época [700/100], Loss: 0.0883
Época [800/100], Loss: 0.0805
Época [900/100], Loss: 0.0754
Época [1000/100], Loss: 0.0719


# Loop de treino com otimizador manual

In [10]:
for epoch in range(1000):
    optimizer.zero_grad()            # Limpa gradientes
    outputs = clone(X)               # Forward
    loss = loss_function(outputs, y) # Calcula perda
    loss.backward()                  # Calcula derivadas do gradiente

    with torch.no_grad():
        for param in clone.parameters():
            param -= learning_rate * param.grad  # regra de atualização de pesos

    if (epoch + 1) % 100 == 0:
        print(f"Época [{epoch+1}/1000], Loss: {loss.item():.4f}")

Época [100/1000], Loss: 1.1878
Época [200/1000], Loss: 0.9264
Época [300/1000], Loss: 0.5037
Época [400/1000], Loss: 0.6505
Época [500/1000], Loss: 1.0960
Época [600/1000], Loss: 1.1328
Época [700/1000], Loss: 0.5926
Época [800/1000], Loss: 0.8987
Época [900/1000], Loss: 1.0938
Época [1000/1000], Loss: 0.6815


# 1. Avaliar se o modelo aprendido está bom

In [11]:
with torch.no_grad():
  outputs_model = model(X)
  predicted_model = (torch.sigmoid(outputs_model) > 0.5).float()
  accuracy_model = (predicted_model == y).float().mean()
  print(f"Acurácia do modelo com SGD: {accuracy_model.item():.4f}")

  outputs_clone = clone(X)
  predicted_clone = (torch.sigmoid(outputs_clone) > 0.5).float()
  accuracy_clone = (predicted_clone == y).float().mean()
  print(f"Acurácia do modelo com otimizador manual: {accuracy_clone.item():.4f}")

Acurácia do modelo com SGD: 0.9867
Acurácia do modelo com otimizador manual: 0.6667


A diferença na acurácia entre os dois modelos se deve à forma como os gradientes são calculados e aplicados durante o treinamento.

O modelo treinado com o `torch.optim.SGD` se beneficia da implementação otimizada do PyTorch para o cálculo e aplicação dos gradientes. O SGD do PyTorch gerencia automaticamente o processo de atualização dos pesos de forma eficiente, garantindo que a regra de atualização (`param -= learning_rate * param.grad`) seja aplicada corretamente a todos os parâmetros do modelo.

Já no loop de treinamento manual, embora a regra de atualização seja a mesma, a forma como ela é aplicada pode ser menos eficiente ou estar sujeita a pequenos detalhes de implementação que impactam o desempenho. Além disso, o `torch.optim.SGD` pode ter otimizações internas ou lidar com casos específicos de forma mais robusta do que a implementação manual simples.

Em resumo, a diferença de desempenho provavelmente se resume à eficiência e precisão da implementação do SGD no PyTorch em comparação com a aplicação manual da regra de atualização.

# 2. Redução da loss na época 100 de acordo com o aumento de neurônios da camada escondida (hidden_dim)

In [24]:
def train_boosted_model(X, y, input_dim=4, hidden_dim=8, output_dim=1):
  boosted = NeuralNet(input_dim, hidden_dim, output_dim)

  learning_rate_boosted = 0.1

  loss_function_boosted = torch.nn.BCEWithLogitsLoss()
  optimizer_boosted = torch.optim.SGD(boosted.parameters(), lr=learning_rate_boosted)

  for epoch in range(1000):
    optimizer_boosted.zero_grad()
    outputs = boosted(X)
    loss = loss_function_boosted(outputs, y)
    loss.backward()
    optimizer_boosted.step()

    if (epoch + 1) % 100 == 0:
      print(f"Época [{epoch+1}/100], Loss: {loss.item():.4f}")

In [26]:
train_boosted_model(X, y, 4, 9, 1)

Época [100/100], Loss: 0.5277
Época [200/100], Loss: 0.3862
Época [300/100], Loss: 0.2704
Época [400/100], Loss: 0.1808
Época [500/100], Loss: 0.1229
Época [600/100], Loss: 0.0920
Época [700/100], Loss: 0.0901
Época [800/100], Loss: 0.0725
Época [900/100], Loss: 0.0687
Época [1000/100], Loss: 0.0670


In [27]:
train_boosted_model(X, y, 4, 10, 1)

Época [100/100], Loss: 0.4963
Época [200/100], Loss: 0.4640
Época [300/100], Loss: 0.3997
Época [400/100], Loss: 0.3993
Época [500/100], Loss: 0.2259
Época [600/100], Loss: 0.1479
Época [700/100], Loss: 0.1149
Época [800/100], Loss: 0.0959
Época [900/100], Loss: 0.0911
Época [1000/100], Loss: 0.0777


In [28]:
train_boosted_model(X, y, 4, 11, 1)

Época [100/100], Loss: 0.5335
Época [200/100], Loss: 0.4412
Época [300/100], Loss: 0.3694
Época [400/100], Loss: 0.3130
Época [500/100], Loss: 0.2524
Época [600/100], Loss: 0.1877
Época [700/100], Loss: 0.1409
Época [800/100], Loss: 0.1118
Época [900/100], Loss: 0.0932
Época [1000/100], Loss: 0.0871


In [31]:
train_boosted_model(X, y, 4, 12, 1)

Época [100/100], Loss: 0.5794
Época [200/100], Loss: 0.5686
Época [300/100], Loss: 0.5558
Época [400/100], Loss: 0.5473
Época [500/100], Loss: 0.5397
Época [600/100], Loss: 0.5336
Época [700/100], Loss: 0.5285
Época [800/100], Loss: 0.5092
Época [900/100], Loss: 0.3995
Época [1000/100], Loss: 0.3099


In [30]:
train_boosted_model(X, y, 4, 13, 1)

Época [100/100], Loss: 0.4601
Época [200/100], Loss: 0.3486
Época [300/100], Loss: 0.3023
Época [400/100], Loss: 0.2312
Época [500/100], Loss: 0.1582
Época [600/100], Loss: 0.1213
Época [700/100], Loss: 0.0969
Época [800/100], Loss: 0.0873
Época [900/100], Loss: 0.0827
Época [1000/100], Loss: 0.0805


In [32]:
train_boosted_model(X, y, 4, 14, 1)

Época [100/100], Loss: 0.5352
Época [200/100], Loss: 0.3847
Época [300/100], Loss: 0.2493
Época [400/100], Loss: 0.1440
Época [500/100], Loss: 0.0939
Época [600/100], Loss: 0.0842
Época [700/100], Loss: 0.0785
Época [800/100], Loss: 0.0724
Época [900/100], Loss: 0.0690
Época [1000/100], Loss: 0.0665


Analisando os resultados do treinamento com diferentes números de neurônios na camada escondida (de 9 a 14), podemos observar a perda (Loss) na época 100 para cada configuração:

- `hidden_dim = 8` (modelo original): Loss na época 100: 0.4795
- `hidden_dim = 9`: Loss na época 100: 0.5277
- `hidden_dim = 10`: Loss na época 100: 0.4963
- `hidden_dim = 11`: Loss na época 100: 0.5335
- `hidden_dim = 12`: Loss na época 100: 0.5794
- `hidden_dim = 13`: Loss na época 100: 0.4601
- `hidden_dim = 14`: Loss na época 100: 0.5352

Não há uma tendência clara de redução monotônica da perda na época 100 à medida que o número de neurônios na camada escondida aumenta. Os valores de perda na época 100 variam entre as diferentes configurações, e em alguns casos, o aumento do número de neurônios resultou em uma perda maior nessa época específica.

É importante notar que a perda na época 100 é apenas um instantâneo do processo de treinamento. O desempenho final do modelo após 1000 épocas pode ser um indicador melhor do impacto do número de neurônios. No entanto, com base apenas na perda na época 100, não podemos afirmar que aumentar os neurônios na camada escondida sempre leva a uma perda menor nesse ponto do treinamento. A otimização de hiperparâmetros, como o número de neurônios, geralmente requer experimentação e avaliação do desempenho final do modelo.